In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import requests
import re
# import urllib.parse
from urllib.parse import urljoin, parse_qsl, unquote, urlparse

import os
import logging
from datetime import datetime

import threading

# PDF 저장 디렉토리와 로그 디렉토리 설정
PDF_DIR = "data/pdf_docs"
LOG_DIR = "data/logs"

# 디렉토리 생성
os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# 로그 설정
log_file = os.path.join(LOG_DIR, f"pdf_download_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file, encoding='utf-8'),
        # logging.StreamHandler()
    ]
)



In [2]:
# WebDriver 인스턴스를 관리하는 싱글톤 클래스
class ChromeDriverWrapper:
    """Chrome WebDriver를 관리하는 싱글톤 클래스"""
    _instance = None
    _driver = None
    _lock = threading.Lock()
    
    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = cls()
        return cls._instance
    
    def get_driver(self):
        if self._driver is None:
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            
            self._driver = webdriver.Chrome(
                service=Service(ChromeDriverManager().install()),
                options=chrome_options
            )
        return self._driver
    
    def quit_driver(self):
        if self._driver:
            try:
                self._driver.quit()
            except Exception as e:
                logging.error(f"드라이버 종료 중 오류: {str(e)}")
            finally:
                self._driver = None

In [3]:

def get_pdf_view_url(driver, cell):
    """PDF 링크를 클릭하고 URL을 반환하는 함수"""
    main_window = None
    try:
        # a 태그 존재 여부 확인
        if not cell.find_elements(By.TAG_NAME, 'a'):
            logging.info("PDF 뷰어 링크(a태그)가 없는 셀입니다.")
            return 'X'
            
        # PDF 링크 찾기 및 클릭
        try:
            pdf_link = cell.find_element(By.CSS_SELECTOR, "a.btn-file.icon-pdf")
            main_window = driver.current_window_handle
            driver.execute_script("arguments[0].click();", pdf_link)
            time.sleep(1)
            
            if len(driver.window_handles) > 1:
                driver.switch_to.window(driver.window_handles[-1])
                url = driver.current_url
                driver.close()
                driver.switch_to.window(main_window)
                if url and url != "about:blank":
                    logging.info(f"PDF 뷰어 URL: {url}")
                return url
            return 'X'
            
        except Exception as e:
            logging.error(f"PDF 뷰어 링크 처리 중 오류: {str(e)}")
            return 'X'
            
    except Exception as e:
        logging.error(f"PDF 뷰어 URL 획득 중 오류: {str(e)}")
        return 'X'
    finally:
        # 창 전환 중 오류 발생한 경우를 대비한 처리
        if main_window:
            try:
                driver.switch_to.window(main_window)
            except Exception as e:
                logging.error(f"PDF 뷰어에서 메인 창 복귀 중 오류: {str(e)}")

   
def get_pdf_download_url(viewer_url):
    """
    PDF 뷰어 URL에서 실제 PDF 파일의 다운로드 URL을 추출하는 함수
    
    인수:
        viewer_url (str): PDF 뷰어 URL 문자열
    
    반환값:
        str or None: PDF 다운로드 URL, 실패시 None
    """
    if not viewer_url or viewer_url == 'X':
        return None
    
    driver_wrapper = ChromeDriverWrapper.get_instance()
    driver = driver_wrapper.get_driver()
    
    try:
        wait = WebDriverWait(driver, 10)
        
        # 이전 페이지 리소스 정리
        try:
            driver.execute_script("window.stop();")
        except:
            pass
            
        driver.get(viewer_url)
        
        # URL 타입 1: CustomerPage_Unit.jsp
        if 'CustomerPage_Unit.jsp' in viewer_url:
            iframe = wait.until(EC.presence_of_element_located((By.ID, 'viewerFrame')))
            src = iframe.get_attribute('src')
            
            if src:
                iframe_url = urljoin('https://pcms.samsunglife.com', src)
                driver.get(iframe_url)
                
                page_source = driver.page_source
                patterns = [
                    r'"filepath"\s*:\s*[\'"](.+?\.pdf)[\'"]',
                    r'"downloadURL"\s*:\s*[\'"](.+?\.pdf)[\'"]'
                ]
                
                for pattern in patterns:
                    match = re.search(pattern, page_source)
                    if match:
                        pdf_path = match.group(1)
                        return urljoin('https://pcms.samsunglife.com', pdf_path)
                
        # URL 타입 2: CustomerPage_Corp.jsp
        elif 'CustomerPage_Corp.jsp' in viewer_url:
            page_source = driver.page_source
            patterns = [
                r'"filepath"\s*:\s*[\'"](.+?\.pdf)[\'"]',
                r'"downloadURL"\s*:\s*[\'"](.+?\.pdf)[\'"]'
            ]
            
            for pattern in patterns:
                match = re.search(pattern, page_source)
                if match:
                    pdf_path = match.group(1)
                    return urljoin('https://pcms.samsunglife.com', unquote(pdf_path))
            
            params = dict(parse_qsl(urlparse(viewer_url).query))
            if 'path' in params and 'fname' in params:
                path = unquote(params['path'])
                fname = unquote(params['fname'])
                return urljoin('https://pcms.samsunglife.com', f"{path}{fname}")
        
        return None
            
    except Exception as e:
        logging.error(f"URL 처리 중 오류 발생: {viewer_url}, 오류: {str(e)}")
        # 오류 발생 시 드라이버 재시작
        driver_wrapper.quit_driver()
        return None
            

def create_filename_from_url(viewer_url, doc_type_name):
    """
    다양한 URL 패턴에서 파일명을 생성하는 함수
    
    Args:
        viewer_url (str): PDF 문서 브라우져 뷰어를 위한 URL
        doc_type_name (str): 문서 종류명 (예: 요약서, 방법서, 약관)
    
    Returns:
        str: 생성된 파일명 또는 None
    """
    if viewer_url == 'X' or not viewer_url or not isinstance(viewer_url, str):
        logging.warning(f"유효하지 않은 URL: {viewer_url}")
        return None
        
    try:
        # URL 파싱
        if 'pcms.samsunglife.com' not in viewer_url:
            logging.warning("삼성생명 도메인이 아닙니다")
            return None
            
        # URL 타입 1: CustomerPage_Unit.jsp 스타일
        if 'CustomerPage_Unit.jsp' in viewer_url:
            params = dict(param.split('=') for param in viewer_url.split('?')[1].split('&'))
            if all(k in params for k in ['goodsCode', 'docType', 'saleDate']):
                return f"{params['goodsCode']}_{params['docType']}_{params['saleDate']}_{doc_type_name}.pdf"
                
        # URL 타입 2: CustomerPage_Corp.jsp 스타일
        elif 'CustomerPage_Corp.jsp' in viewer_url and 'path=' in viewer_url and 'fname=' in viewer_url:
            params = dict(param.split('=') for param in viewer_url.split('?')[1].split('&'))
            
            # path에서 날짜 정보 추출
            path_param = params.get('path', '')
            path_parts = path_param.split('/')
            if len(path_parts) >= 5:
                year = path_parts[-4]
                month_day = path_parts[-3]
                sale_date = f"{year}{month_day}"
                doc_type = path_parts[-2]  # 701 또는 801
                
                # 1순위: fname에서 상품코드 추출
                fname_param = params.get('fname', '')
                product_code = fname_param.split('-')[0]
                
                # 상품코드가 의미있는 값인 경우 (숫자로만 구성되지 않았거나, 길이가 짧은 경우)
                if not (product_code.isdigit() and len(product_code) >= 10):
                    return f"{product_code}_{doc_type}_{sale_date}_{doc_type_name}.pdf"
                    
                # 2순위: title 파라미터가 있는 경우
                # 기존 로직에서 title 파라미터 처리 부분 개선
                if 'title' in params and params['title'].strip():
                    # URL decoding 적용             
                    product_name = unquote(params['title'].strip())
                    # 파일명에 사용할 수 없는 문자 제거
                    product_name = re.sub(r'[\\/:*?"<>|]', '', product_name)
                    return f"{product_name}_{doc_type}_{sale_date}_{doc_type_name}.pdf"            
                                    
                # 3순위: 현재 시간의 타임스탬프 사용
                from datetime import datetime
                timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
                return f"{timestamp}_{doc_type}_{sale_date}_{doc_type_name}.pdf"
            
        # URL 타입 3: 기타 패턴
        else:
            if '/uploadDir/doc/' in viewer_url:
                parts = viewer_url.split('/uploadDir/doc/')[1].split('/')
                if len(parts) >= 5:
                    year = parts[0]
                    month_day = parts[1]
                    goods_code = parts[2]
                    doc_type = parts[3]
                    return f"{goods_code}_{doc_type}_{year}{month_day}_{doc_type_name}.pdf"
                    
        logging.warning(f"지원되지 않는 URL 형식: {viewer_url}")
        return None
        
    except Exception as e:
        logging.error(f"파일명 생성 중 오류 발생: {str(e)}, URL: {viewer_url}")
        # 오류 발생 시 타임스탬프로 파일명 생성
        try:
            from datetime import datetime
            timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
            return f"{timestamp}_ERROR_{doc_type_name}.pdf"
        except:
            return None
            
def download_pdf(viewer_url, file_name, doc_type_name, save_dir):
    """
    PDF 뷰어 URL에서 실제 PDF를 다운로드하여 저장하는 함수
    
    Args:
        viewer_url (str): PDF 뷰어 URL
        file_name (str): 저장할 파일명
        doc_type_name (str): 문서 유형 (예: "요약서", "약관")
        save_dir (str): 저장할 디렉토리 경로
    """
    if not viewer_url or not file_name:
        logging.error(f"{doc_type_name} - 유효하지 않은 URL 또는 파일명")
        return
        
    # 파일 존재 여부 확인
    file_path = os.path.join(save_dir, file_name)
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path)
        if file_size > 0:
            logging.info(f"{doc_type_name} - 파일이 이미 존재합니다: {file_name}")
            return
        else:
            # 크기가 0인 파일은 삭제하고 다시 다운로드
            os.remove(file_path)
            logging.info(f"{doc_type_name} - 빈 파일이 존재하여 삭제 후 다시 다운로드합니다: {file_name}")
        
    try:
        # 실제 PDF 다운로드 URL 획득
        pdf_url = get_pdf_download_url(viewer_url)
        if not pdf_url:
            logging.error(f"{doc_type_name} 다운로드 PDF URL 획득 실패")
            return
            
        session = requests.Session()
        headers = {
            'Accept': 'application/pdf,application/x-pdf,application/octet-stream',
            'Accept-Language': 'ko-KR,ko;q=0.9',
            'Connection': 'keep-alive',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/131.0.0.0 Safari/537.36',
            'Referer': viewer_url
        }
        
        # PDF 파일 다운로드 시도
        response = session.get(pdf_url, headers=headers, stream=True, timeout=30)
        response.raise_for_status()
        
        # Content-Type 확인
        content_type = response.headers.get('Content-Type', '').lower()
        if not any(ct in content_type for ct in ['application/pdf', 'application/octet-stream']):
            logging.error(f"{doc_type_name} - 잘못된 Content-Type: {content_type}")
            return
            
        # 파일 저장
        file_path = os.path.join(save_dir, file_name)
        total_size = int(response.headers.get('content-length', 0))
        
        with open(file_path, 'wb') as f:
            if total_size == 0:
                f.write(response.content)
            else:
                downloaded = 0
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        downloaded += len(chunk)
                        # 다운로드 진행률 표시
                        progress = (downloaded / total_size) * 100
                        if progress % 20 == 0:  # 20% 단위로 로그 출력
                            logging.info(f"{doc_type_name} 다운로드 진행률: {progress:.1f}%")
                            
        # 파일 크기 확인
        if os.path.getsize(file_path) > 0:
            logging.info(f"{doc_type_name} 다운로드 성공: {file_name}")
        else:
            logging.error(f"{doc_type_name} 다운로드 실패: 파일 크기가 0")
            os.remove(file_path)  # 빈 파일 삭제
            
    except requests.exceptions.RequestException as e:
        logging.error(f"{doc_type_name} 다운로드 중 네트워크 오류: {str(e)}")
    except Exception as e:
        logging.error(f"{doc_type_name} 다운로드 중 오류 발생: {str(e)}")

In [4]:
def scrape_samsung_life_products_on_sale(driver):
    data0 = []
    wait = WebDriverWait(driver, 30)

    try:
        driver.get("https://www.samsunglife.com/individual/products/disclosure/sales/PDO-PRPRI010110M")
        logging.info("삼성생명 공시실 상품공시 웹사이트에 접속했습니다.")
        # 페이지 완전 로딩 대기
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "tabs-group")))
        time.sleep(5)  # 추가 대기 시간

        tabs_to_process = ['판매상품', '판매중지상품']
        tab_content_map = {
            '판매상품': 'content1',
            '판매중지상품': 'content2'
        }

        # tabs_to_process = ['판매상품']
        # tab_content_map = {
        #     '판매상품': 'content1'
        # }
        
        for tab_name in tabs_to_process:
            logging.info(f"\n{tab_name} 탭 처리 시작...")
            
            try:
                # 탭 선택 및 컨텐츠 로드
                tab = wait.until(EC.element_to_be_clickable((
                    By.XPATH, 
                    f"//ul[@class='tabs-group']/li/a[normalize-space()='{tab_name}']"
                )))
                driver.execute_script("arguments[0].click();", tab)
                time.sleep(1)

                content_id = tab_content_map[tab_name]
                
                while True:  # 페이지 순환
                    try:
                        # 현재 페이지의 tbody 찾기
                        tbody_xpath = f"//div[@id='{content_id}']//tbody[@class='line']"
                        tbody = wait.until(EC.presence_of_element_located((By.XPATH, tbody_xpath)))
                        rows = tbody.find_elements(By.TAG_NAME, 'tr')
                        logging.info(f"현재 페이지에서 {len(rows)}개의 행을 찾았습니다.")

                        # 현재 페이지의 데이터 처리
                        for row in rows:
                            try:
                                cells = row.find_elements(By.TAG_NAME, 'td')
                                # 판매중지상품은 6개, 판매상품은 7개의 셀이 필요
                                min_cells = 7 if tab_name == "판매상품" else 6
                                if len(cells) < min_cells:
                                    logging.warning(f"셀 개수 부족: {len(cells)}")
                                    continue

                                record_num = cells[0].text.strip()
                                
                                # 판매상품 탭의 경우 데이터 수집
                                if tab_name == "판매상품":
                                    summary_url = get_pdf_view_url(driver, cells[4])
                                    file_name_summary = 'X'
                                    if summary_url != 'X':
                                        file_name_summary = create_filename_from_url(summary_url, "요약서")
                                        download_pdf(summary_url, file_name_summary, "요약서", PDF_DIR)

                                    method_url = get_pdf_view_url(driver, cells[5])
                                    file_name_method = 'X'
                                    if method_url != 'X':
                                        file_name_method = create_filename_from_url(method_url, "방법서")
                                        download_pdf(method_url, file_name_method, "방법서", PDF_DIR)

                                    terms_url = get_pdf_view_url(driver, cells[6])
                                    file_name_terms = 'X'
                                    if terms_url != 'X':
                                        file_name_terms = create_filename_from_url(terms_url, "약관")
                                        download_pdf(terms_url, file_name_terms, "약관", PDF_DIR)
                                    
                                    current_record = [
                                        record_num,
                                        "삼성생명",
                                        "판매중",
                                        cells[1].text.strip(),
                                        cells[2].find_element(By.CSS_SELECTOR, '.title-link, a').text.strip(),
                                        cells[3].text.strip(),
                                        file_name_summary,
                                        file_name_method,
                                        file_name_terms,
                                        summary_url,
                                        method_url,
                                        terms_url
                                    ]
                                else:
                                    # 판매중지상품 탭의 경우
                                    method_url = get_pdf_view_url(driver, cells[4])
                                    file_name_method = 'X'
                                    if method_url != 'X':
                                        file_name_method = create_filename_from_url(method_url, "방법서")
                                        download_pdf(method_url, file_name_method, "방법서", PDF_DIR)

                                    terms_url = get_pdf_view_url(driver, cells[5])
                                    file_name_terms = 'X'
                                    if terms_url != 'X':
                                        file_name_terms = create_filename_from_url(terms_url, "약관")
                                        download_pdf(terms_url, file_name_terms, "약관", PDF_DIR)

                                    current_record = [
                                        record_num,
                                        "삼성생명",
                                        "판매중지",
                                        cells[1].text.strip(),
                                        cells[2].find_element(By.CSS_SELECTOR, '.title-link, a').text.strip(),
                                        cells[3].text.strip(),
                                        'X',  # 요약서 항상 'X'
                                        file_name_method,
                                        file_name_terms,
                                        'X',  # 요약서viewer 항상 'X'
                                        method_url,
                                        terms_url
                                    ]
                                
                                data0.append(current_record)
                                # print(f"레코드 {record_num} 처리 완료")

                            except Exception as e:
                                logging.error(f"행 처리 중 오류: {e}")
                                continue

                        # 페이지네이션 처리
                        pagination = wait.until(EC.presence_of_element_located((
                            By.XPATH, f"//div[@id='{content_id}']//div[@class='pagination-number']"
                        )))
                        
                        # 현재 페이지 번호 확인
                        current_page_text = pagination.find_element(By.CSS_SELECTOR, "li.current button").text
                        current_page = int(current_page_text.split()[0].strip())
                        logging.info(f"현재 페이지: {current_page} 페이지 수집완료...")

                        # 다음 마지막 페이지 여부 확인 부분 수정
                        try:
                            last_set_button = wait.until(
                                EC.presence_of_element_located((
                                    By.XPATH,
                                    f"//div[@id='{content_id}']//button[@class='btn-paging-last']"
                                ))
                            )
                            # disabled 속성의 존재 여부 확인
                            is_last_set = last_set_button.get_attribute('disabled') is not None
                        except Exception as e:
                            logging.error(f"마지막 페이지 버튼 확인 중 오류: {e}")
                            is_last_set = False

                        if is_last_set:
                            # 마지막 세트의 마지막 페이지인 경우
                            logging.info(f"{tab_name} 탭의 모든 데이터 수집 완료")
                            time.sleep(2)  # 탭 전환 전 잠시 대기
                            break  # while 루프 종료

                        # 다음 페이지 세트 버튼 확인
                        next_set_button = wait.until(
                            EC.presence_of_element_located((
                                By.XPATH,
                                f"//div[@id='{content_id}']//button[@class='btn-paging-next']"
                            ))
                        )

                        if current_page % 10 == 0:  # 10, 20, 30 등 페이지 세트의 마지막
                            if is_last_set:
                                # 마지막 세트의 마지막 페이지인 경우
                                logging.info(f"{tab_name} 탭의 모든 데이터 수집 완료")
                                time.sleep(2)  # 탭 전환 전 잠시 대기
                                break  # while 루프 종료
                            
                            logging.info("다음 페이지 세트로 이동")
                            old_tbody = tbody
                            driver.execute_script("arguments[0].click();", next_set_button)
                            time.sleep(2)
                            
                            # 다음 세트의 첫 번째 페이지로 이동
                            first_page_of_set = ((current_page // 10) * 10) + 1
                            first_page_xpath = f"//div[@id='{content_id}']//div[@class='pagination-number']//li/button[text()='{first_page_of_set}']"
                            
                            try:
                                first_page_button = wait.until(
                                    EC.presence_of_element_located((By.XPATH, first_page_xpath))
                                )
                                driver.execute_script("arguments[0].click();", first_page_button)
                                time.sleep(2)
                                continue
                            except Exception as e:
                                logging.error(f"다음세트 첫 페이지 이동 중 오류: {e}")
                                break
                        else:
                            # 일반적인 다음 페이지 이동
                            try:
                                next_page = current_page + 1
                                next_button_xpath = f"//div[@id='{content_id}']//div[@class='pagination-number']//li[not(@class='current')]/button[text()='{next_page}']"
                                
                                try:
                                    next_button = wait.until(EC.presence_of_element_located((By.XPATH, next_button_xpath)))
                                    logging.info(f"페이지 {next_page}로 이동")
                                    driver.execute_script("arguments[0].click();", next_button)
                                    time.sleep(2)
                                    continue
                                except TimeoutException:
                                    # 다음 버튼을 찾지 못한 경우
                                    if is_last_set:
                                        logging.info(f"{tab_name}의 마지막 페이지입니다. 다음 탭으로 이동합니다.")
                                        break
                                    else:
                                        logging.info("다음 페이지 세트로 이동이 필요합니다.")
                                        continue
                                    
                            except Exception as e:
                                if is_last_set:
                                    logging.info(f"{tab_name}의 마지막 페이지입니다. 다음 탭으로 이동합니다.")
                                    break
                                else:
                                    logging.error(f"페이지 이동 중 오류 발생: {e}")
                                    break

                    except Exception as e:
                        logging.error(f"페이지 처리 중 오류: {e}")
                        if is_last_set:
                            logging.info(f"{tab_name} 탭의 모든 데이터 수집 완료")
                            time.sleep(2)  # 탭 전환 전 잠시 대기
                            break  # while 루프 종료
                        break

            except Exception as e:
                logging.error(f"탭 처리 중 오류: {e}")
                continue  # 다음 탭으로 이동

            # 현재 탭의 데이터 수집이 완료되면 진행 상황 출력
            logging.info(f"\n=== {tab_name} 탭 처리 완료 ===")
            logging.info(f"수집된 데이터 수: {len(data0)}")
            if tab_name != tabs_to_process[-1]:  # 마지막 탭이 아닌 경우
                logging.info(f"다음 탭 '{tabs_to_process[tabs_to_process.index(tab_name) + 1]}' 처리 시작...\n")
                time.sleep(3)  # 탭 전환 전 충분한 대기 시간

    except Exception as e:
        logging.error(f"크롤링 중 오류 발생: {e}")

    finally:
        return pd.DataFrame(data0, columns=[
            "품번", "판매사", "판매구분", "분류", "상품명", 
            "판매기간", "요약서", "방법서", "약관", "요약서viewer", "방법서viewer", "약관viewer"
        ]), driver

In [5]:
def create_chrome_driver():
    """Chrome WebDriver 생성 및 설정"""
    chrome_options = Options()
    
    # 리눅스 환경 특화 설정
    chrome_options.add_argument('--headless=new')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--remote-debugging-port=9222')  # 디버깅 포트 추가
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--disable-setuid-sandbox')
    chrome_options.add_argument('--single-process')  # 단일 프로세스 모드
    
    # 기본 설정
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--start-maximized')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument('--enable-javascript')
    
    # PDF 다운로드 설정
    pdf_download_path = os.path.abspath("data/pdf_docs")
    os.makedirs(pdf_download_path, exist_ok=True)
    
    prefs = {
        "download.default_directory": pdf_download_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True,
        "safebrowsing.enabled": True,
        "profile.default_content_settings.popups": 0
    }
    chrome_options.add_experimental_option("prefs", prefs)
    
    # 자동화 감지 방지
    chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        return driver
    except Exception as e:
        logging.error(f"Chrome 드라이버 생성 실패: {str(e)}")
        raise
    

try:
    # Chrome 드라이버 생성
    driver = create_chrome_driver()
    logging.info("Chrome 드라이버 생성 성공")
    
    # 스크래핑 실행
    df1, driver = scrape_samsung_life_products_on_sale(driver)
    
    if len(df1) > 0:
        output_file = "samsung_life_products.xlsx"
        df1.to_excel(output_file, index=False)
        logging.info(f"데이터를 {output_file} 파일로 저장했습니다.")
        logging.info(f"총 {len(df1)} 개의 행이 저장되었습니다.")
    else:
        logging.error("수집된 데이터가 없습니다.")

except Exception as e:
    logging.error(f"프로그램 실행 중 오류 발생: {str(e)}")
    
finally:
    if 'driver' in locals():
        driver.quit()
        logging.info("브라우저를 종료했습니다.")